In [ ]:
# Cell 1: Install required packages (run once)
# Adjust / remove if already installed
!pip install -q pandas tqdm transformers sentencepiece sacremoses fasttext langdetect sentence_transformers rapidfuzz accelerate
!pip install -q "huggingface_hub>=0.17.0"

# Optional high-quality Indic translation (if you want to try later)
# !pip install -q git+https://github.com/AI4Bharat/indic-trans.git

In [ ]:
# Cell 2: Imports and config
import os, glob, json, re
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
from rapidfuzz import fuzz
from rapidfuzz import process as rfp
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Make sure GPU is available
DEVICE = 0 if torch.cuda.is_available() else -1
print("Using device:", "cuda:0" if DEVICE==0 else "cpu")

In [ ]:
# Cell 3: User-config - folders and files

RAW_FOLDER = "/kaggle/input/finetuningdataforagroshakti"     # Folder containing all CSV and parquet files
OUT_FOLDER = "/kaggle/working/cleaned_output"
os.makedirs(OUT_FOLDER, exist_ok=True)

# Glob input CSVs for CropQueryDataset files
csv_files = sorted(glob.glob(os.path.join(RAW_FOLDER, "*CropQueryDataset*.csv")))
# parquet_files = sorted(glob.glob(os.path.join(RAW_FOLDER, "*CropQueryDataset*.parquet")))

print("Found CSV files:", len(csv_files))
for f in csv_files:
    print("  ", os.path.basename(f))

In [ ]:
# Cell 4: Columns selection and standardization
# We'll map incoming columns to canonical names and only keep the selected ones
keep_cols = {
    "StateName": "state",
    "DistrictName": "district",
    "Category": "category",
    "Crop": "crop",
    "QueryType": "query_type",
    "QueryText": "query",
    "KccAns": "answer",
    "year": "year",
    "month": "month"
}


In [ ]:
# Cell 5: Load and concat CSVs (with basic cleaning of column names)
dfs = []
for file in csv_files:
    df = pd.read_csv(file, dtype=str, keep_default_na=False, na_values=["", "NA", "None"], engine='python',
    on_bad_lines='skip')
    # Normalize column names
    df.columns = [c.strip() for c in df.columns]
    # Keep only available mapping columns
    available = {k:v for k,v in keep_cols.items() if k in df.columns}
    df = df[list(available.keys())].rename(columns=available)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True, sort=False)
print("Combined rows:", len(df))

In [ ]:
df.head()

In [ ]:
# Cell 6: Quick stats & drop rows with missing query/answer
print("Before dropna:", len(df))
df = df[df['query'].notna() & (df['query'].str.strip() != "")]
df = df[df['answer'].notna() & (df['answer'].str.strip() != "")]
print("After dropping null queries/answers:", len(df))

In [ ]:
# Cell 7: Text cleaning helper functions
# Regex-based cleaning, keep language-aware punctuation
def basic_clean_text(text):
    if not isinstance(text, str):
        return ""
    t = text.strip()
    # remove odd unicode like zero-width etc
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    # Replace multiple spaces/newlines
    t = re.sub(r'\s+', ' ', t)
    # Remove URLs and emails
    t = re.sub(r'http\S+|www\.\S+|[\w\.-]+@[\w\.-]+', '', t)
    # Remove long numeric IDs etc
    t = re.sub(r'\b\d{6,}\b', '', t)
    # Remove emojis and non-text symbols (keep basic punctuation)
    t = re.sub(r'[^\w\s\.\,\?\!\-\/\u0900-\u097F]', '', t)  # allow Devanagari range
    t = t.strip(" -,.")
    return t

# Normalize whitespace & punctuation for queries & answers
df['query'] = df['query'].map(basic_clean_text)
df['answer'] = df['answer'].map(basic_clean_text)

In [ ]:
df.info()

In [ ]:
len(df)

In [ ]:
# Cell 8: Remove obvious non-agri rows via keyword blacklist (extend as needed)
# This is conservative: remove rows containing bank/account/complaint terms
non_agri_keywords = [
    "bank", "loan", "atm", "mobile recharge", "complaint", "registration", "adhar", "aadhar",
    "helpline number", "phone is not working", "electricity bill", "police", "payment", "hanged",
    "HANGED", "BANK", "LOAN"
]
pattern = re.compile("|".join(re.escape(k) for k in non_agri_keywords), flags=re.IGNORECASE)
df = df[~df['query'].str.contains(pattern, na=False)]
print("After non-agri filter:", len(df))

In [ ]:
!pip install datasketch

In [ ]:
# Cell 10: Deduplication - exact then fast near-duplicate dedupe using MinHash LSH
from datasketch import MinHash, MinHashLSH

# Fast MinHash + LSH dedupe (replaces fuzzy dedupe)
def minhash_lsh_dedupe(df, text_col='query', threshold=0.92, num_perm=64):
    """
    threshold: similarity threshold (0.8–0.95 recommended)
    num_perm: number of hash permutations (64 = fast, 128 = more accurate)
    """
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    keep_idx = []
    minhashes = {}

    for idx, text in tqdm(df[text_col].fillna("").items(),
                          total=len(df), desc="MinHash LSH Dedupe"):
        # Create MinHash
        m = MinHash(num_perm=num_perm)
        for word in str(text).lower().split():
            m.update(word.encode('utf8'))

        # Check if similar already exists
        result = lsh.query(m)
        if not result:         # no near-duplicate found → keep this row
            lsh.insert(idx, m)
            keep_idx.append(idx)

    return df.loc[keep_idx]


# Run fast dedupe
before = len(df)
df = minhash_lsh_dedupe(df, text_col='query', threshold=0.95, num_perm=64)
print("Removed near-duplicates:", before - len(df))
print("After dedupe:", len(df))

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

In [ ]:
import fasttext
import tqdm
import pandas

# Load fastText language ID model
lang_model = fasttext.load_model("lid.176.bin")

def detect_lang_fast(text):
    if not isinstance(text, str) or len(text.strip()) < 3:
        return "unknown"

    # Detect Indic scripts first (more accurate than model on romanized text)
    if re.search(r'[\u0900-\u097F]', text): return 'hi'   # Hindi/Marathi/Nepali
    if re.search(r'[\u0A00-\u0A7F]', text): return 'pa'   # Punjabi
    if re.search(r'[\u0980-\u09FF]', text): return 'bn'   # Bengali
    if re.search(r'[\u0C80-\u0CFF]', text): return 'kn'   # Kannada

    # FastText expects at least 3 characters of text
    try:
        pred = lang_model.predict(text.replace("\n", " ")[:200])[0][0]
        return pred.replace("__label__", "")  # example: "__label__en" -> "en"
    except:
        return "unknown"


tqdm.pandas()
df['a_lang'] = df['answer'].progress_apply(detect_lang_fast)

In [ ]:
df = df[df['a_lang'].isin(['hi','en'])]

print('new size',len(df))

print(df['a_lang'].unique())
print(df['a_lang'].nunique())

In [ ]:
!pip install IndicTransToolkit huggingface_hub transformers --quiet

In [ ]:
from huggingface_hub import login
login('')

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor

# -------------------------------
# 1. Load your dataset
# -------------------------------
# Change 'your_dataset.csv' to your actual file name
df = pd.read_csv("/kaggle/working/filtered_halfpreprocessed.csv")

# Make sure there's a column named 'answer'
if "answer" not in df.columns:
    raise ValueError("Dataset must have a column named 'answer'")

# -------------------------------
# 2. Setup translation model
# -------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Hindi → English model
model_name = "ai4bharat/indictrans2-indic-en-1B"
src_lang, tgt_lang = "hin_Deva", "eng_Latn"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).to(DEVICE)


In [ ]:
ip = IndicProcessor(inference=True)

# -------------------------------
# 3. Define translation function
# -------------------------------
def translate_hindi_to_english(sentence):
    if not isinstance(sentence, str) or sentence.strip() == "":
        return ""

    try:
        # detect language
        lang = detect(sentence)

        if lang != "hi":  # If not Hindi, return as is
            return sentence

        # preprocess
        batch = ip.preprocess_batch([sentence], src_lang=src_lang, tgt_lang=tgt_lang)
        inputs = tokenizer(batch, truncation=True, padding="longest", return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=False,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        generated_tokens = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        translation = ip.postprocess_batch(generated_tokens, lang=tgt_lang)[0]
        return translation
    except Exception as e:
        print(f"Error translating: {sentence}\n{e}")
        return sentence

# -------------------------------
# 4. Apply translation on the dataset
# -------------------------------
df["translated"] = df["answer"].apply(translate_hindi_to_english)

# -------------------------------
# 5. Save the result
# -------------------------------
df.to_csv("translated_dataset.csv", index=False)
print("✅ Translation completed. Saved as translated_dataset.csv")

In [ ]:
!pip install truecase

In [ ]:
import truecase

df['state'] = df['state'].apply(lambda x: truecase.get_true_case(x))
df['district'] = df['district'].apply(lambda x: truecase.get_true_case(x))
df['query'] = df['query'].apply(lambda x: truecase.get_true_case(x))

In [ ]:
!pip install IndicTransToolkit --quiet

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL = "ai4bharat/indictrans2-indic-en-dist-200M"

tok = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL, trust_remote_code=True,
    torch_dtype=torch.float16, attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

def translate_df(df, state, src_lang, batch_size):
    mask = (df["state"] == state) & (df["a_lang"] == "hi")
    texts = df.loc[mask, "answer"].tolist()
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc=state):
        batch = ip.preprocess_batch(
            texts[i:i+batch_size], src_lang=src_lang, tgt_lang="eng_Latn"
        )
        inp = tok(batch, padding="longest", truncation=True, return_tensors="pt")
        inp = {k: v.to(DEVICE, non_blocking=True) for k, v in inp.items()}

        with torch.no_grad():
            out = model.generate(**inp, num_beams=1, max_length=128,
                                 no_repeat_ngram_size=2, early_stopping=True)

        trans = ip.postprocess_batch(
            tok.batch_decode(out, skip_special_tokens=True), lang="eng_Latn"
        )
        results.extend(trans)

        del inp, out, batch, trans
        torch.cuda.empty_cache()

    df.loc[mask, "answer"] = results


# ------------------ CONFIG ------------------
configs = [
    ("Haryana", "pan_Guru", 32),
    ("West Bengal", "ben_Beng", 64),
    ("Karnataka", "kan_Knda", 32),
    ("Punjab", "pan_Guru", 128),
]

for state, lang, bs in configs:
    translate_df(df, state, lang, bs)


In [ ]:
mask = (df['state'] == 'Punjab') & (df['a_lang'] == 'hi')
df.loc[mask, 'translated'] = df.loc[mask, 'answer']

In [ ]:
min_tokens = 6
df = df[df['translated'].str.split().str.len() >= min_tokens]

In [ ]:
df.drop(columns=['answer', 'year', 'a_lang'], inplace=True)

In [ ]:
df.rename(columns={'translated': 'answer'}, inplace=True)

In [ ]:
cols = ['query_type', 'query', 'answer']

for col in cols:
    df[col] = df[col].astype(str).str.strip()             # remove leading/trailing spaces
    df[col] = df[col].str.replace(r'\s+', ' ', regex=True)  # replace multiple spaces with single space

In [ ]:
df = df[df['query_type'] != 'Weather']

In [ ]:
for qt, count in df['query_type'].value_counts().items():
    print(f"{qt} ------ {count}")

In [ ]:
x = df['query_type'].unique()
for i in x:
    print(i)

In [ ]:
crops = df['crop'].unique()
for c in crops:
    count = df['crop'].value_counts().loc[c]
    print(f"{c} ------ {count}")


In [ ]:
df.to_csv('/kaggle/working/translated_dataset_final6.csv')